<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# Constrained Scheduling

## Intro

In this Notebook we will formulate an optimization problem with discrete variables. Programatically we will use the [Constrained Quadratic Model](https://docs.ocean.dwavesys.com/en/latest/concepts/cqm.html) (CQM) class in Ocean tools to formulate it.

Then we will use DWave Hybrid solvers to sample the DQM object of our optimization problem. Specifically we will use the [LeapHybridCQMSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridcqmsampler) and the SAPI to sumbmit the sampling job to the QPU

### DWave libs set-up

We install DWave Ocean Libraries and setup them.

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
!dwave setup

In [ ]:
!dwave config inspect

## Constrained Quadratic Models


The constrained quadratic model (CQM) are problems of the form:



*   Minimize an objective:

$\sum_{i=1} a_i x_i + \sum_{i=1} \sum_{j=1} b_{i,j} (x_i x_j)+c$

*   Subject to constrains:

$\sum_{i=1} a_i^{(m)} x_i + \sum_{i=1} \sum_{j=1} b_{i,j}^{(m)} (x_i x_j)+c^{(m)} \circ 0, \ \ \ m= 1,..,M$

where:
- $\{x_i\}_{x=1,..N}$ can be binary $^{[1]}$, integer, or continous $^{[2]}$ variables
- $a_i$ and $b_{i,j}$ are real-values
- $c$ is a real value constant (offset)
- $\circ \in \{\leq, \geq, = \}$ are possible operations
- and $M$ the total number of constrains

$^{[1]}$ For binary variables, the range of the quadratic-term summation is $i<j$ because $x^2=x$
 for binary values $\{0,1\}$ and $s^2=1$ for spin values $\{-1,1\}$

$^{[2]}$ Real-valued variables are currently not supported in quadratic interactions.

The [ConstrainedQuadraticModel](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/models.html#dimod.ConstrainedQuadraticModel) class can contain this model and its methods provide convenient utilities for working with representations of a problem.

## CSP Employee scheduling

Building a schedule for employees can be an extremely complex optimization problem in which managers must balance employee preferences against schedule requirements. In this example, we show how a constrain quadratic model (CQM) can be used to model this problem and how the hybrid solvers available in DWave Leap can optimize over these competing scheduling and preference needs.

The CQM formulation has several advantages as including constrains staraightforwadly in the model with no guess for Lagrange parameters and a more concise programming syntax for formulating constrains.

First we import the libraries needed in the notebook

In [ ]:
from dimod import ConstrainedQuadraticModel, Binary, quicksum
from dwave.system import LeapHybridCQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt

To make the example more felxible we ask the user to provide the input values of the number of employees $M$ and the number of shifts $N$.

Note that we should have at least as many employees as there are shifts.

In [ ]:
# Collect user input on size of problem
response_1 = input("\nEnter number of employees > ")
try:
    num_employees = int(response_1)
except ValueError:
    print("Must input an integer.")
    num_employees = int(input("\nEnter number of employees > "))

response_2 = input("\nEnter number of shifts > ")
try:
    num_shifts = int(response_2)
except ValueError:
    print("Must input an integer.")
    num_shifts = int(input("\nEnter number of shifts > "))

if num_employees < num_shifts:
    print("\n**Number of employees must be at least number of shifts.**")

    print("\nEnter number of employees:")
    num_employees = int(input())
    print("\nEnter number of shifts:")
    num_shifts = int(input())

print("\nScheduling", num_employees, "employees over", num_shifts, "shifts...\n")

Another input data are the preferences of each employee for the shifts. In order to make things simpler the program will randomly generate employee preferences for the N shifts from most preferred (0) to least preferred (N-1).

In [ ]:
# Generate random array of preferences over employees
preferences = np.tile(np.arange(num_shifts), (num_employees, 1))
rows = np.indices((num_employees,num_shifts))[0]
cols = [np.random.permutation(num_shifts) for _ in range(num_employees)]
preferences = preferences[rows, cols]
print(preferences)

### Formulate the problem: Building the CQM

The employee scheduling problem consists of two components: a requirement that employees are evenly distributed across all shifts, and an objective to satisfy employees by scheduling them for their preferred shifts.

To build the model we construct a Discrete Quadratic Models object [(see CQM class)](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridcqmsampler) and add it a set of $N$ `Binary` variables for each employee, that indicates the employee's shift.

These `Binary` variables are added to the model as $M$ discrete variables that can take up to $N$ different values.

In [ ]:
# Initialize the CQM object
cqm = ConstrainedQuadraticModel()

# Build the CQM starting by creating variables
vars = [[Binary(f'x_{name}_{i}') for i in range(num_shifts)] for name in range(num_employees)]

# Add constraint to make variables discrete
for v in range(len(vars)):
    cqm.add_discrete([f'x_{v}_{i}' for i in range(num_shifts)])

In [ ]:
print(cqm)

#### Preferred Shifts

Since shift preferences $p_{i,u}$ are ranked from most preferred (smallest value) to least preferred (largest value), the rankings can be used for the biases in the quadratic model. Since the hybrid solvers look for minimum energy solutions, and minimum rank corresponds to most preferred, these naturally align.

The objective function of our problem can be formulatrd as:

$\sum_i^M \sum_u^N p_{i,u}x_{i,u}$

To formulate it, we use the [`quicksum` tool](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/generated/dimod.binary.quicksum.html) provided in `dimod` libraries, that is optimized to be faster than the python method `sum()`

In [ ]:
# Objective: maximize employee preference (choose shifts with lower preference numbers)
obj = quicksum([preferences[j,i]*vars[j][i] for j in range(num_employees) for i in range(num_shifts)])
cqm.set_objective(obj)

In [ ]:
print(cqm)

#### Even Distribution

An even distribution of employees across shifts would have approximately num_employees/num_shifts $M/N$ scheduled employees per shift. To enforce this requirement, both linear and quadratic biases must be adjusted in a specific manner. We can do this using linear and quadratic biases to enforce the constraint in the discrete quadratic model.

To determine the linear and quadratic bias adjustments, we must consider the underlying binary variables in our model. For a DQM with $N$ shifts and $M$ employees, each employee has a single variable $d_i$ constructed with $N$ cases or classes. These are implemented as $N$ binary variables $x_{i,u}$ per employee — one for each possible shift.

For a specific shift $u$, we require that exactly $M/N$ employees are scheduled. This is equivalent to saying that $M/N$ employee variables are assigned case $u$, or, returning to our binary variables, that $M/N$ of the binary variables corresponding to case $u$ take a value of 1. In other words, the sum of all employee case $u$ binary variables should equal $M/N$. An equality over a summation of binary variables can be converted to a minimization expression by moving from the equality:

$\sum_{i=1}^M x_{i,u} = \frac{M}{N}  \ \ \ \ \
\forall u$

In the CQM we can include constrains in a straightforward maner with the method `.addconstrain()`, as it allows algebraic mathematical formulation format.

In [ ]:
# Constraint: equal number of employees per shift
num_per_shift = int(num_employees/num_shifts)
for i in range(num_shifts):
    cst = quicksum([vars[j][i] for j in range(num_employees)])
    cqm.add_constraint(cst == num_per_shift, label=f'c_{i}')

In [ ]:
print(cqm)

### Solve the problem in a QPU with Leap Hybrid Solvers

To sample a CQM we use a DWave Hybrid solver, the [LeapHybridCQMSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridcqmsampler) and use the SAPI to sumbmit the sampling job to the QPU

In [ ]:
# Initialize the CQM solver
sampler = LeapHybridCQMSampler()

# Solve the problem using the DQM solver
sampleset = sampler.sample_cqm(cqm, label='Example - Employee Scheduling')

We print the sampling results

In [ ]:
#get all solutions
print("\nSchedule first sample:", sampleset)

In [ ]:
# Get the first feasible solution
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)
if len(feasible_sampleset) == 0:
    print("\nNo feasible solution found. Returning best infeasible solution.")
    sample = sampleset.first.sample
    energy = sampleset.first.energy
else:
    sample = feasible_sampleset.first.sample
    energy = feasible_sampleset.first.energy
print("\nSample:", sample)
print("\nSchedule score:", energy)

### Final results

Once we have the results, we build two images to ilustrate the distibution of the employees in the shifts:

- First, `employee_schedule.png` illustrates the employee preference matrix alongside the schedule built.
- Second, `schedule_statistics.png` shows how many employees are scheduled for each shift, alongside a bar chart showing the employees' average preferences for the shifts for which they have been scheduled.

At the end of the program's run, two statistics are printed to the command-line:

- Schedule score provides the energy value of the best solution found.
- Average happiness is the average of the employee preference values for the shifts that they are scheduled.

A smaller average happiness score is better, and a score of 0.0 is a perfect score - everyone received their first choice shift.

In [ ]:
# Build schedule
schedule = np.ones((num_employees, num_shifts))*num_shifts
prefs = [0]*num_shifts
shifts = [0]*num_shifts
for key, val in sample.items():
    if val == 1:
        v = key.split("_")
        emp = int(v[1])
        shft = int(v[2])
        schedule[emp,shft]=preferences[emp,shft]
        prefs[preferences[emp,shft]] += 1
        shifts[shft] += 1

# Show heatmap of preferences
cmap = plt.get_cmap('seismic')
cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist[-1] = (1.0,1.0,1.0,1.0)
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(preferences, cmap='seismic', interpolation='nearest', vmin=0, vmax=num_shifts, aspect='auto')
ax1.xlabel = 'Shifts'
ax1.ylabel = 'Employees'
ax1.set_title("Employee Shift Preferences", color='Black', fontstyle='italic')

# Show heatmap of schedule
cax = ax2.imshow(schedule, cmap=cmap, interpolation='nearest', aspect='auto', vmin=0)
cbar = fig.colorbar(cax, ticks=[0, num_shifts])
cbar.set_ticklabels(['Best', 'Worst'])
ax2.xlabel = 'Shifts'
ax2.set_title("Employee Shift Schedule", color='Black', fontstyle='italic')
plt.savefig("employee_schedule.png")

# Compute/display schedule statistics
plt.clf()
plt.subplot(1, 2, 1)
plt.bar(np.arange(num_shifts), shifts)
plt.xlabel("Shift")
plt.ylabel("Number Scheduled")
plt.title("Employees Scheduled Per Shift")

mean_happiness = sum([i*prefs[i] for i in range(num_shifts)])/num_employees
print("\nAverage happiness:\t", mean_happiness)

plt.subplot(1, 2, 2)
plt.bar(np.arange(num_shifts), prefs)
plt.xlabel("Preference Rank")
plt.title("Average Preference per Shift")
plt.savefig("schedule_statistics.png")